# Import stuff

In [9]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.odr import *
from scipy import stats
from matplotlib import rcParams
from ipynb.fs.full.redshift import get_data

# Define functions

In [10]:
def linear(p, x): # fit a line
    return p[1] * x + p[0]

def power(p, x): # fit a power law
    return p[0] * x ** p[1]

# Get data

In [27]:
data = np.genfromtxt('main.csv', delimiter = ',', dtype = None, names = True)
# print(data.dtype.names)

lotss_flux       = data['lofTotal_flux'          ]
lotss_flux_error = data['lofE_Total_flux_tot'    ]
nvss_flux        = data['NVSS_Jy'                ]
nvss_flux_error  = data['NVSS_error_Jy'          ]
source_type1     = data['lacOpticalClass'        ]
source_type2     = data['bzcSourceclassification']

lotss_flux       = [i / 1000 for i in lotss_flux      ] # convert to Jansky
lotss_flux_error = [i / 1000 for i in lotss_flux_error] # convert to Jansky

source_type = []

for i in range(len(source_type1)): # take classification from 3LAC in the first instance and BZCAT after
    if str(source_type1[i])[2:-2] == 'bll':
        source_type.append('BLLac')
    elif str(source_type1[i])[2:-2] == 'fsrq':
        source_type.append('FSRQ')
    elif len(str(source_type1[i])[2:-2]) > 0:
        source_type.append('other')
    elif str(source_type2[i])[2:-2] == 'BL Lac':
        source_type.append('BLLac')
    elif  str(source_type2[i])[2:-2] == 'QSO RLoud flat radio sp.':
        source_type.append('FSRQ')
    else:
        source_type.append('other')
        
BLLac_MHz, BLLac_MHz_error, BLLac_GHz, BLLac_GHz_error = [], [], [], []
FSRQ_MHz,  FSRQ_MHz_error,  FSRQ_GHz,  FSRQ_GHz_error  = [], [], [], []
other_MHz, other_MHz_error, other_GHz, other_GHz_error = [], [], [], []

for i in range(len(lotss_flux)):
    if source_type[i] == 'BLLac':
        BLLac_MHz.append(lotss_flux[i])
        BLLac_MHz_error.append(lotss_flux_error[i])
        BLLac_GHz.append(nvss_flux[i])
        BLLac_GHz_error.append(nvss_flux_error[i])
    elif source_type[i] == 'FSRQ':
        FSRQ_MHz.append(lotss_flux[i])
        FSRQ_MHz_error.append(lotss_flux_error[i])
        FSRQ_GHz.append(nvss_flux[i])
        FSRQ_GHz_error.append(nvss_flux_error[i])
    else:
        other_MHz.append(lotss_flux[i])
        other_MHz_error.append(lotss_flux_error[i])
        other_GHz.append(nvss_flux[i])
        other_GHz_error.append(nvss_flux_error[i])

# Find the line of best fit
I use SciPy's [orthogonal distance regression](https://docs.scipy.org/doc/scipy/reference/odr.html).

In [28]:
lotss_weight, nvss_weight = [], []

for i in range(len(lotss_flux)):
    lotss_weight.append(lotss_flux[i] / (lotss_flux_error[i]))
    nvss_weight.append(nvss_flux[i] / (nvss_flux_error[i]))
    
model = Model(linear)
data = Data(np.log(lotss_flux), np.log(nvss_flux), wd = lotss_weight, we = nvss_weight)
odr = ODR(data, model, beta0 = [-1, 0.8])
odr.set_job(fit_type = 0)
out = odr.run()
# out.pprint()

cov_x_y = out.cov_beta[0][1]
var_x = out.cov_beta[0][0]
var_y = out.cov_beta[1][1]
r2 = (cov_x_y ** 2) / (var_x * var_y)
r = np.sqrt(r2)
N = len(lotss_flux)
t = r / np.sqrt((1 - r2)/(N - 2))
p = stats.t.sf(np.abs(t), N - 1) * 2

print('r =', round(r, 4), 'r^2 =', round(r2, 4), 'N =', N, 't =', round(t, 4), 'p =', '{:.2e}'.format(p))

out.beta[0] = np.exp(out.beta[0])

lotss_fit = np.linspace(min(lotss_flux), max(lotss_flux), 1000)
nvss_fit = power(out.beta, lotss_fit)

r = 0.7664 r^2 = 0.5873 N = 98 t = 11.6894 p = 3.20e-20


# Make the plot

In [38]:
plot = False

color = 'black'
line = [1e-5, 1e3]

if plot:
    color = 'white'

plt.rcParams['font.family'] = 'serif'
plt.figure(figsize = (13.92, 8.60))

plt.plot(line, line, linestyle = '--', color = 'grey', linewidth = 2)
plt.errorbar(BLLac_MHz, BLLac_GHz, xerr = BLLac_MHz_error, yerr = BLLac_GHz_error, linestyle = 'None', marker = 'o', markersize = 8, color = 'red', label = 'BL Lacs')
plt.errorbar(FSRQ_MHz, FSRQ_GHz, xerr = FSRQ_MHz_error, yerr = FSRQ_GHz_error, linestyle = 'None', marker = '^', markersize = 8, color = 'blue', label = 'FSRQs')
plt.errorbar(other_MHz, other_GHz, xerr = other_MHz_error, yerr = other_GHz_error, linestyle = 'None', marker = 's', markersize = 8, color = 'green', label = 'Other')
#plt.errorbar(lotss_flux, nvss_flux, xerr = lotss_flux_error, yerr = nvss_flux_error, linestyle = 'None', marker = 'x', markersize = 2, color = 'black')
plt.plot(lotss_fit, nvss_fit, color = 'black', linewidth = 2)

plt.xlim(9e-4, 2e1)
plt.ylim(9e-4, 2e1)

plt.xscale('log')
plt.yscale('log')
plt.xlabel(r'$S_{\mathrm{MHz}}$ (Jy)', fontsize = 30, color = color)
plt.ylabel(r'$S_{\mathrm{GHz}}$ (Jy)', fontsize = 30, color = color)
plt.xticks(fontsize = 30, color = color)
plt.yticks(fontsize = 30, color = color)

legend = plt.legend(bbox_to_anchor = (0, 1.0, 1, 0), loc = 'lower left', mode = 'expand', numpoints = 1, fontsize = 30, ncol = 4, frameon = False)
plt.setp(legend.get_texts(), color = color)

plt.annotate('1', (0.0012768962941999999, 0.023623499999999999), ha = 'center', fontsize = 30)
plt.annotate('2', (0.0025004658568000002, 0.053153499999999999), ha = 'center', fontsize = 30)
plt.annotate('3', (0.0079077353842999989, 0.095895499999999995), ha = 'center', fontsize = 30)
plt.annotate('4', (0.0302021013897, 0.00355514), ha = 'center', fontsize = 30)

if not plot:
    plt.savefig('radio_vs_radio.pdf', bbox_inches = 'tight', format = 'pdf')
elif plot:
    plt.show()